# Simplex Method

## Introduction

In mathematical optimization, simplex method is a popular linear programming algorithm that can solve many linear optimization problems. Optimization problems aim to select the best element regarding certain criterias among all possible elements. To select the best element, simplex method introduces slack variables, tableaus, and pivot variables to find the optimal solution for an optimization problem. The simplex technique can be solved with python codes and this artifact will explain how a standard simplex method work in solving problems with an example illustrated.

### Procedure of Simplex Method
Consider the linear optimization problem: maximize $\mathbf{c}^T \mathbf{x}$ subject to $A \mathbf{x} \leq \mathbf{b}$, $\mathbf{x} \geq 0$. Right here we have an objective function of maximizing $\mathbf{c}^T \mathbf{x}$, the decision variable $\mathbf{x}$ and constraints  $A \mathbf{x} \leq \mathbf{b}$, $\mathbf{x} \geq 0$. To perform simplex method, we need to rewrite the function and make it into a standard form and then:

- Introduce slack variables for each constraint
- Create tableau
- Pivot variables
- Use pivot to generate new tableau
- Check for optimality
- Repeat pivot if necessary
- Check for optimal Values


## Case Study 1 -- Standard Simplex Method

In this section, we design a matrix and use the pivot function to solve the problem as each pivot is equivalent to the vertex. We change pivots until it gets to an optimization.

Now we consider using the random-generated simplex problems tool from the Vanderbei textbook:

$$
\begin{array}{rl}
\text{maximize:} & -2x_1 − x_2 + x_3 \\ 
\text{subject to:} & x_1 + 4x_3 \leq 4 \\
& -x_1 + x_2 + x_3 \leq 5 \\
& x_1, x_2, x_3 \geq 0 \\
\end{array}
$$

### Question: How does the simplex method work?

In [3]:
import numpy as np
import scipy.linalg as la
from scipy.optimize import linprog

Rewrite the linear problem into matrix form:

$$
\mathbf{A} = \begin{bmatrix} 1 & 0 & 4 \\ -1 & 1 & 1 \end{bmatrix}
\hspace{1in}
\mathbf{b} = \begin{bmatrix} 4 \\ 5 \end{bmatrix}
\hspace{1in}
\mathbf{c} = \begin{bmatrix} -2 \\ -1 \\ 1 \end{bmatrix}
$$

#### Create Tableau and Introduce Slack Variables

In [4]:
A = np.array([[1.,0.,4.],[-1.,1.,1.]])
m,n = A.shape
I = np.eye(m)
c = np.array([-2.,-1.,1.])
b = np.array([4.,5.])
T = np.vstack([np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[ 1.  0.  4.  1.  0.  4.]
 [-1.  1.  1.  0.  1.  5.]
 [-2. -1.  1.  0.  0.  0.]]


Basically we put rewrited matrices listed above to a large matrix where all the information is contained in it. The first two rows are constraints and the last row is the objective function.
The matrix T looks like the following in math notations if we compared the matrix T values with matrix A, b and c:

$$
T = 
\begin{bmatrix}
A & I & \mathbf{b} \\
\mathbf{c}^T & 0 & 0
\end{bmatrix}
$$

where I is an identity matrix.

Firstly, notice that we introduce two columns (column 4 and column 5) as slack variables. The reason to add slack variables is to change the inequality function to equality. The question has two constraints thus we need 2 slack variables. each row corresponds to an equation. Each slack variable will always has a +1 coefficient in its particular constraint. Since slack variables are required in the constraints to transform them into solvable equalities with one definite answer.

The last row is the objective function. The goal is to maximize the last row, where (-2,-1,1) are the coefficients for basic variables. The first two zeros in the last row (0,0) are slack variables, which are not in the original objective function of this problem. Thus, we set the values for slack variables to 0. The last 0 in this row is the value of this objective function.

#### Choose Pivot
The purpose of pivot is to adjust the tableau by selecting an entering variable and a leaving variable to an adjacent vertex for optimality. 

For choosing pivot, we have to consider the coefficients on the last row of the tableau. We aim to choose the largest positive coefficient among all the x to increase the value of this variable for potential optimal value of this objective function, also denote as the entering variable. After choosing pivot, we have to choose the leaving variable. We check the particular column of the pivot. Then, we use the corresponding constraint value divided by the matched value for each constraint in the pivot column, and compare the results. We choose the smallest value as a leaving variable.

Then we can perform pivot transformation to generate a new tableau to check the optimality.

In [5]:
def pivot(T,k,l):
    E = np.eye(T.shape[0])
    E[:,l] = -T[:,k]/T[l,k]
    E[l,l] = 1/T[l,k]
    return E@T
T1 = pivot(T,2,0)

Here is a function for pivot-shifting. It takes the tableau T we created above, a value k and a value $l$. In mathematics, k is called entering variable and $l$  is called leaving variable.

The value k is the corresponding column of our chosen pivot. In this case, we would like to choose $x_3$ to be the pivot and its corresponding column is when k=2. 

Continue on analyzing the last row of this matrix. The largest number among all x variables is 1 among (-2,-1,1,0,0), thus we choose it as a pivot. After choosing it, we look at the its column(column 3). Now we compare $4 \div 4 = 1$ and $5 \div 1 = 5$. We pick the smaller value as the leaving varialbe for changing the pivots. Then the leaving variable here should be the first row, where $l$=0.

#### Generate New Tableau and Check Optimality

In [6]:
print(T1)

[[ 0.25  0.    1.    0.25  0.    1.  ]
 [-1.25  1.    0.   -0.25  1.    4.  ]
 [-2.25 -1.    0.   -0.25  0.   -1.  ]]


Here is a new tableau and we are going to check its optimality. We see all the numbers in the last row is less or equal to zero, which means all the coefficients of decision variables are non-negative. Since the pivot definiton results negative values if we maximize the objective function, if all values on the last row are non-positive, we have reached an optimal. Therefore, the solution has an optimal value of 1 for this particular problem.

## Case Study 2 -- Degeneracy

### Theory
Think of the cases when the basic solution doesn't change. Sometimes we are not able to choose a leaving variable where at least one of the basic variables is zero. When things happen like that, we call it degenerate as no leaving variable will not give any improvement to the objective function value. Degeneracy does not bring actual change in the result but only changes the basic variables in the model (Degeneracy, 2022). Degeneracy would occur in some real-life applications but it usually have little harm to the solving the problem.


### Degeneracy Example
Now consider an example of degeneracy. 


$$
\begin{array}{rl}
\text{maximize:} & x_1 +2x_2 \\ 
\text{subject to:} & x_1 +x_2 \leq 3 \\
& x_2 \leq 2 \\
& 1/2x_1 + x_2 \leq 2.5 \\
& x_1, x_2 \geq 0
\end{array}
$$


Rewrite the linear problem into matrix form:

$$
\mathbf{A} = \begin{bmatrix} 1 & 1 \\ 0 & 1 \\ 1/2 & 1 \end{bmatrix}
\hspace{1in}
\mathbf{b} = \begin{bmatrix} 3 \\ 2 \\ 2.5 \end{bmatrix}
\hspace{1in}
\mathbf{c} = \begin{bmatrix} 1 \\ 2 \end{bmatrix}
$$

#### Create Tableau and Introduce Slack Variables

In [49]:
A = np.array([[1.,1.],[0.,1.],[1/2,1.]])
m,n = A.shape
I = np.eye(m)
c = np.array([1.,2.])
b = np.array([3.,2.,2.5])
T = np.vstack([np.hstack([A,I,b.reshape((m,1))]) , np.hstack([c,np.zeros(m+1)]) ])
print(T)

[[1.  1.  1.  0.  0.  3. ]
 [0.  1.  0.  1.  0.  2. ]
 [0.5 1.  0.  0.  1.  2.5]
 [1.  2.  0.  0.  0.  0. ]]


#### Perform Pivoting
Then we perform some pivot-shifting and see if degeneracy occurs in the following tableaus.

In [50]:
T1 = pivot(T,1,1)
print(T1)

[[ 1.   0.   1.  -1.   0.   1. ]
 [ 0.   1.   0.   1.   0.   2. ]
 [ 0.5  0.   0.  -1.   1.   0.5]
 [ 1.   0.   0.  -2.   0.  -4. ]]


Then if we continue to apply the pivoting rules for standard simplex method, then the entering variable is k=0. However, if we look at the columns and decide to choose a leaving variable, we notice that $1\div{1}$ and $0.5\div{0.5}$ output the same result. This case is called degeneracy because we have more than one possible leaving variables.

In [51]:
T2 = pivot(T1,0,0)
print(T2)

[[ 1.   0.   1.  -1.   0.   1. ]
 [ 0.   1.   0.   1.   0.   2. ]
 [ 0.   0.  -0.5 -0.5  1.   0. ]
 [ 0.   0.  -1.  -1.   0.  -5. ]]


In [52]:
T3 = pivot(T1,0,2)
print(T3)

[[ 0.  0.  1.  1. -2.  0.]
 [ 0.  1.  0.  1.  0.  2.]
 [ 1.  0.  0. -2.  2.  1.]
 [ 0.  0.  0.  0. -2. -5.]]


If we compare T2 and T3 tableaus, we can see that the values of the objective function are exactly the same. Because we have ties in choosing leaving variable. Since we do not have any improvements for the optimal, such action cannot make any progresses to optimality. 

The worse case of degeneracy is when the same set of basic variables occurs twice. In this case, the pivoting method in the simplex algorithm will produce an infinite loop called "cycle" (Camarena).

### Bland's Rule -- Prevent Degeneracy

We introduce bland's rule to prevent degeneracy if we have situations like above. Bland's rule chooses the row with the lowest numbered variables in the basis. It means that:


1. When choosing an entering variable, we should choose the the first one in the ordering that has a positive entry in the objective row.

In [54]:
print(T)

[[1.  1.  1.  0.  0.  3. ]
 [0.  1.  0.  1.  0.  2. ]
 [0.5 1.  0.  0.  1.  2.5]
 [1.  2.  0.  0.  0.  0. ]]


In the case above, we choose $x_1$ because the the entry is 1. There is no tie in leaving varaible, so we proceed.

In [57]:
T1 = pivot(T,1,1)

2. When choosing a leaving variable and there is a tie for the least ratio, take the first one in the ordering.

In [58]:
print(T1)

[[ 1.   0.   1.  -1.   0.   1. ]
 [ 0.   1.   0.   1.   0.   2. ]
 [ 0.5  0.   0.  -1.   1.   0.5]
 [ 1.   0.   0.  -2.   0.  -4. ]]


In the case above, we still the first column as entering varialbe according to Bland's rule, but there is a tie in ratio for the column. The values are correspond to slack variables $s_3$ and $s_5$. Convert s to x to match the notation for all decison variables then we have $x_3$ and $x_5$. Bland's rule suggests to choose the first one in the ordering for tied least ratio. In this case, we choose $x_3$, and proceed pivoting.

In [59]:
T2 = pivot(T1,0,0)
print(T2)

[[ 1.   0.   1.  -1.   0.   1. ]
 [ 0.   1.   0.   1.   0.   2. ]
 [ 0.   0.  -0.5 -0.5  1.   0. ]
 [ 0.   0.  -1.  -1.   0.  -5. ]]


Now this tableau reaches optimality and we call it end. Thus, the optimal solutions equals to 5.

Bland's rule can help the occurance of degeneracy and prevent looping, which is a very useful rule.

## Conclusion

The purpose of using matrix form to solve could save lots of time when we increase number of constraints and variables. In reality, some of the problems are much more complicated than that and might require multiple pivot-shiftings. The method produces an optimal solution to satisfy the given constraints and produce a maximum objective when all the coefficients on the decision variables are non-negative. To use the simplex method, the model should be linearr and in standard form, where we introduce slack variables then. To proceed, we use the tableau and pivot to check optimality until an optimal solution is reached.

Sometimes special case like degeneracy will happen to affect the results and we can apply bland's rule to prevent it. Degeneracy will waste time for calculation or it will become a cycle in worse cases. In general, it is not efficient for the simplex method model but might be occur in various real-world cases. Now we have a method to deal with the degeneracy, which makes simplex method a very strong linear programming algorithm.

## References

Camarena, O. A. (n.d.). Bland’s Rule guarantees termination. Bland's rule guarantees termination. Retrieved December 8, 2022, from https://www.matem.unam.mx/~omar/math340/blands-rule.html 

Wikimedia Foundation. (2022, November 19). Simplex algorithm. Wikipedia. Retrieved December 8, 2022, from https://en.wikipedia.org/wiki/Simplex_algorithm


Wikimedia Foundation. (2022, November 29). Degeneracy. Wikipedia. Retrieved December 8, 2022, from https://en.wikipedia.org/wiki/Degeneracy
